# 0. Configuration

In [1]:
data_dir = 'data/'
graph_filename = 'deezer_europe_edges.csv'

graph_path = f'{data_dir}/{graph_filename}'
sep = ','
directed = False
header = True

In [2]:
import time

# 1. Implementation of graph-traversal algorithms from scratch

In [3]:
from collections import defaultdict, deque

### 1.1 Data loading
We will load the graph adopting an **adjacency list** representation.

Specifically, we will use a **dictionary** where the key is a vertex identifier, and the value is the set of vertex identifiers of all the neighbors of the key vertex.

In [4]:
def load_graph(graph_path,sep,directed,header):
    with open(graph_path) as f:
        start = 1 if header else 0
        vertex2neighbors = defaultdict(set)
        for line in f.readlines()[start:]:
            tokens = line.split(sep)
            vertex1 = int(tokens[0])
            vertex2 = int(tokens[1])
            vertex2neighbors[vertex1].add(vertex2)
            if not directed:
                vertex2neighbors[vertex2].add(vertex1) #for undirected graphs, every edge is stored twice; can we do better?
    return vertex2neighbors

In [5]:
start = time.time()
graph = load_graph(graph_path,sep,directed,header)
end = time.time()
runtime = int(round((end-start)*1000))
print("Loading time: " + str(runtime) + " ms")

Loading time: 110 ms


### 1.2 Auxiliary functions

In [6]:
def number_of_vertices(graph):
    return len(graph)

def number_of_edges(graph,directed):
    denominator = 1 if directed else 2
    return sum([len(graph[u]) for u in graph.keys()])/denominator # '/2' is needed for undirected graphs because every edge is stored twice

def neighborhood(graph,vertex):
    return graph[vertex]

### 1.3 Breadth-first search (BFS)
<img src="./img/pseudocode_bfs.png" align="left" width="70%"/>

In [7]:
def bfs(graph, source):
    visited = {source} # Python set 
    L = [source] # Python list
    Q = deque([source]) # queue implemented as a 'deque' class from the 'collections' Python built-in module
    while Q:
        currently_selected = Q.popleft() # 'dequeue' operation
        for u in neighborhood(graph,currently_selected):
            if u not in visited:
                visited.add(u)
                L.append(u)
                Q.append(u) # 'enqueue' operation
    return L

#### 1.3.1 BFS on 'deezer_europe' dataset

In [8]:
source = 0

start = time.time()
L = bfs(graph,source)
end = time.time()
runtime = int(round((end-start)*1000))
print("BFS time: " + str(runtime) + " ms")

len(L), number_of_vertices(graph) # sanity check

BFS time: 35 ms


(28281, 28281)

#### 1.3.2 BFS on 'toy' dataset
<img src="./img/toy.png" align="left" width="35%"/>

In [9]:
graph_filename_toy = 'toy.csv'
graph_path_toy = f'{data_dir}/{graph_filename_toy}'
graph_toy = load_graph(graph_path_toy,sep,directed,header)
L0 = bfs(graph_toy,0)
L3 = bfs(graph_toy,3)
L7 = bfs(graph_toy,7)
print("BFS from source vertex 0: " + str(L0)) #0 1 2 3 7 4 5 6
print("BFS from source vertex 3: " + str(L3)) #3 1 2 4 5 0 7 6
print("BFS from source vertex 7: " + str(L7)) #7 2 4 0 1 3 5 6

BFS from source vertex 0: [0, 1, 2, 3, 7, 4, 5, 6]
BFS from source vertex 3: [3, 1, 2, 4, 5, 0, 7, 6]
BFS from source vertex 7: [7, 2, 4, 0, 1, 3, 5, 6]


### 1.4 Depth-first Search
<img src="./img/pseudocode_dfs.png" align="left" width="70%"/>

In [10]:
def dfs(graph, source):
    visited = set()
    L = []
    S = deque([source])
    while S:
        currently_selected = S.pop()
        if currently_selected not in visited:
            visited.add(currently_selected)
            L.append(currently_selected)
            for u in neighborhood(graph,currently_selected):
                if u not in visited:
                    S.append(u)
    return L

#### 1.4.1 DFS on 'deezer_europe' dataset

In [11]:
source = 0

start = time.time()
L = dfs(graph,source)
end = time.time()
runtime = int(round((end-start)*1000))
print("DFS time: " + str(runtime) + " ms")

len(L), number_of_vertices(graph) # sanity check

DFS time: 56 ms


(28281, 28281)

#### 1.4.2 DFS on 'toy' dataset
<img src="./img/toy.png" align="left" width="35%"/>

In [12]:
graph_filename_toy = 'toy.csv'
graph_path_toy = f'{data_dir}/{graph_filename_toy}'
graph_toy = load_graph(graph_path_toy,sep,directed,header)
L0 = dfs(graph_toy,0)
L3 = dfs(graph_toy,3)
L7 = dfs(graph_toy,7)
print("DFS from source vertex 0: " + str(L0)) #0 2 7 4 6 5 3 1
print("DFS from source vertex 3: " + str(L3)) #3 5 6 4 7 2 1 0
print("DFS from source vertex 7: " + str(L7)) #7 4 6 5 3 2 1 0

DFS from source vertex 0: [0, 2, 7, 4, 6, 5, 3, 1]
DFS from source vertex 3: [3, 5, 6, 4, 7, 2, 1, 0]
DFS from source vertex 7: [7, 4, 6, 5, 3, 2, 1, 0]


### 1.5 Connected components

In [13]:
def connected_components(graph,visit):
    remaining_vertices = set(graph.keys()).copy()
    connected_components = []
    while remaining_vertices:
        source = next(iter(remaining_vertices))
        cc = visit(graph,source)
        connected_components.append(cc)
        for u in cc:
            remaining_vertices.remove(u)
    return connected_components

#### 1.5.1 Connected components of 'toy_multipleCC' dataset
<img src="./img/toy_multipleCC.png" align="left" width="60%"/>

In [14]:
graph_filename_toy_multipleCC = 'toy_multipleCC.csv'
graph_path_toy_multipleCC = f'{data_dir}/{graph_filename_toy_multipleCC}'
graph_toy_multipleCC = load_graph(graph_path_toy_multipleCC,sep,directed,header)

connected_components_bfs = connected_components(graph_toy_multipleCC,bfs)
print("Connected components with BFS traversal: " + str(connected_components_bfs))

connected_components_dfs = connected_components(graph_toy_multipleCC,dfs)
print("Connected components with DFS traversal: " + str(connected_components_dfs))

Connected components with BFS traversal: [[0, 1, 2, 3, 7, 4, 5, 6], [8, 9, 10, 13, 11, 12], [14, 17, 15, 16], [18, 19], [20, 21]]
Connected components with DFS traversal: [[0, 2, 7, 4, 6, 5, 3, 1], [8, 10, 11, 13, 9, 12], [14, 15, 17, 16], [18, 19], [20, 21]]


# 2. Graph traversal with [`NetworkX`](https://networkx.org/)

In [15]:
import networkx as nx

### 2.1 Data loading

In [16]:
def load_graph_nx(graph_path,sep,directed,header):
    graph_type = nx.DiGraph if directed else nx.Graph
    with open(graph_path, 'rb') as f:
        if header:
            next(f, '') # skip header line
        G = nx.read_adjlist(f, delimiter=sep, create_using=graph_type, nodetype=int)
    return G

### 2.2 DFS on 'toy' dataset
<img src="./img/toy.png" align="left" width="35%"/>

In [17]:
graph_filename_toy = 'toy.csv'
graph_path_toy = f'{data_dir}/{graph_filename_toy}'
graph_nx_toy = load_graph_nx(graph_path_toy,sep,directed,header)
L0_nx = list(nx.dfs_preorder_nodes(graph_nx_toy, source=0)) # list of the vertices in the order that they were first visited by DFS
L3_nx = list(nx.dfs_preorder_nodes(graph_nx_toy, source=3)) # list of the vertices in the order that they were first visited by DFS
L7_nx = list(nx.dfs_preorder_nodes(graph_nx_toy, source=7)) # list of the vertices in the order that they were first visited by DFS
print("DFS (with NetworkX) from source vertex 0: " + str(L0_nx))
print("DFS (with NetworkX) from source vertex 3: " + str(L3_nx))
print("DFS (with NetworkX) from source vertex 7: " + str(L7_nx))

DFS (with NetworkX) from source vertex 0: [0, 1, 2, 3, 4, 5, 6, 7]
DFS (with NetworkX) from source vertex 3: [3, 1, 0, 2, 7, 4, 5, 6]
DFS (with NetworkX) from source vertex 7: [7, 2, 0, 1, 3, 4, 5, 6]


### 2.3 BFS on 'toy' dataset
<img src="./img/toy.png" align="left" width="35%"/>

In [18]:
bfs_layers_nx_0 = list(nx.bfs_layers(graph_nx_toy, sources=0))
bfs_layers_nx_3 = list(nx.bfs_layers(graph_nx_toy, sources=3))
bfs_layers_nx_7 = list(nx.bfs_layers(graph_nx_toy, sources=7))
print("BFS layers (with NetworkX) from source vertex 0: " + str(bfs_layers_nx_0))
print("BFS layers (with NetworkX) from source vertex 3: " + str(bfs_layers_nx_3))
print("BFS layers (with NetworkX) from source vertex 7: " + str(bfs_layers_nx_7))

BFS layers (with NetworkX) from source vertex 0: [[0], [1, 2], [3, 7], [4, 5], [6]]
BFS layers (with NetworkX) from source vertex 3: [[3], [1, 2, 4, 5], [0, 7, 6]]
BFS layers (with NetworkX) from source vertex 7: [[7], [2, 4], [0, 1, 3, 5, 6]]


In [19]:
# just flattening 'bfs_layers'
bfs_ordering_nx_0 = [x for y in bfs_layers_nx_0 for x in y]
bfs_ordering_nx_3 = [x for y in bfs_layers_nx_3 for x in y]
bfs_ordering_nx_7 = [x for y in bfs_layers_nx_7 for x in y]
print("BFS (with NetworkX) from source vertex 0: " + str(bfs_ordering_nx_0)) #0 1 2 3 7 4 5 6
print("BFS (with NetworkX) from source vertex 3: " + str(bfs_ordering_nx_3)) #3 1 2 4 5 0 7 6
print("BFS (with NetworkX) from source vertex 7: " + str(bfs_ordering_nx_7)) #7 2 4 0 1 3 5 6

BFS (with NetworkX) from source vertex 0: [0, 1, 2, 3, 7, 4, 5, 6]
BFS (with NetworkX) from source vertex 3: [3, 1, 2, 4, 5, 0, 7, 6]
BFS (with NetworkX) from source vertex 7: [7, 2, 4, 0, 1, 3, 5, 6]


### 2.4 Connected components of 'toy_multipleCC' dataset
<img src="./img/toy_multipleCC.png" align="left" width="60%"/>

In [20]:
graph_filename_toy_multipleCC = 'toy_multipleCC.csv'
graph_path_toy_multipleCC = f'{data_dir}/{graph_filename_toy_multipleCC}'
graph_nx_toy_multipleCC = load_graph_nx(graph_path_toy_multipleCC,sep,directed,header)

connected_components_nx = nx.connected_components(graph_nx_toy_multipleCC)
print("Connected components with NetworkX: " + str(list(connected_components_nx)))

Connected components with NetworkX: [{0, 1, 2, 3, 4, 5, 6, 7}, {8, 9, 10, 11, 12, 13}, {16, 17, 14, 15}, {18, 19}, {20, 21}]
